In [44]:
system"T 1"
show lines:read0`test.txt
rawPath:last lines
lines:-2_lines

"        ...#"
"        .#.."
"        #..."
"        ...."
"...#.......#"
"........#..."
"..#....#...."
"..........#."
"        ...#...."
"        .....#.."
"        .#......"
"        ......#."
""
"10R5L5R10L4R5L5"


In [47]:
/ append initial rotation for symmetry - starting from facing down (vector 1 0)
show path:(`$;"J"$)@'/:2 cut (raze 0 1+/:where any path=/:.Q.A) cut path:"L",rawPath
show maxC:max count each lines
show maxR:count lines
show FIELD:(maxC$)@'lines / pad for simplicity

`L 10
`R 5 
`L 5 
`R 10
`L 4 
`R 5 
`L 5 
16
12
"        ...#    "
"        .#..    "
"        #...    "
"        ....    "
"...#.......#    "
"........#...    "
"..#....#....    "
"..........#.    "
"        ...#...."
"        .....#.."
"        .#......"
"        ......#."


In [48]:
R:{(x 1;neg x 0)} / rotate 90 degrees clockwise
L:{(neg x 1;x 0)} / counter

get (`R;1 0)

0 -1


In [49]:
SP:(0;first FIELD[0]?".")
SD:(1 0) / start facing down

move:{[pd;t;s]
    prevp:currp:pd 0;d:get(t;pd 1);
    /show .Q.s1(currp;d;s);
    while[s>0;
        currp:(currp+d)mod(maxR;maxC);
        c:FIELD . currp;
        /show .Q.s1(currp;s;c);
        if["#"=c;:(prevp;d)];
        if["."=c;s-:1;prevp:currp];
     ];
    (currp;d)
 }

move[(SP;SD);`L;4]

0 10
0 1 


In [37]:
test:{[f;i;o]
    if[not o~r:f . i;
        '"expected: ",.Q.s1[o],", but got ",.Q.s1[r]," for ",.Q.s1 i
     ];
    :1b
 }

In [38]:
t1:test move
all t1 .' (
    (((0 8;1 0);`L;1);(0 9;0 1));
    (((0 8;1 0);`L;2);(0 10;0 1));
    (((0 8;1 0);`L;3);(0 10;0 1)); / wall
    (((0 8;0 1);`R;1);(1 8;1 0)); 
    (((0 8;0 1);`R;2);(1 8;1 0)); / wall
    (((3 8;1 0);`L;1);(3 9;0 1));
    (((3 8;1 0);`L;3);(3 11;0 1));
    (((3 8;1 0);`L;4);(3 8;0 1))
 )

1b


In [40]:
/ part 1
facing:(0 1;1 0;0 -1;-1 0)!til 4

show .Q.s1 res:move/[(SP;SD);path[;0];path[;1]]
/ adjust for 0-based index
sum 1000 4 1*1 1 0+res[0],first facing res 1

"(5 7;0 1)"


6032


In [93]:
nextpd:{[pd]
    @[pd;0;+;pd 1]mod(maxR;maxC) / (currp+d)mod(maxR;maxC)
 }

/ recreate the same but with function to produce next position/direction
move:{[pd;t;s]
    currpd:prevpd:@[pd;1;t];
    /show .Q.s1(currp;d;s);
    while[s>0;
        currpd:nextpd[currpd];
        c:FIELD . currpd 0;
        /show .Q.s1(currp;s;c);
        if["#"=c;:prevpd];
        if["."=c;s-:1;prevpd:currpd];
     ];
    currpd
 }

all t1 .' (
    (((0 8;1 0);`L;1);(0 9;0 1));
    (((0 8;1 0);`L;2);(0 10;0 1));
    (((0 8;1 0);`L;3);(0 10;0 1)); / wall
    (((0 8;0 1);`R;1);(1 8;1 0)); 
    (((0 8;0 1);`R;2);(1 8;1 0)); / wall
    (((3 8;1 0);`L;1);(3 9;0 1));
    (((3 8;1 0);`L;3);(3 11;0 1));
    (((3 8;1 0);`L;4);(3 8;0 1))
 )

1b


In [90]:
show N:maxR div 3; / face side - 4 in example, 50 in real

tile:{d:(0 2;1 0;1 1;1 2;2 2;2 3)!1 2 3 4 5 6;d x div N}
t2:test tile
all t2 .' (
    (enlist 5 11;4);
    (enlist 3 11;1);
    (enlist 8 11;5);
    (enlist 8 12;6);
    (enlist 5 7;3);
    (enlist 5 3;2)
 )

4


1b


In [136]:
TS:1 2 3 4 5 6!(0 8;4 0;4 4;4 8;8 8;8 12) / tile starts
nextpd:{[pd]
    p:pd 0;d:pd 1;
    t:tile p;
    0N!p:p mod N;
    / A->B 5 11 -> 8 14 (tile 4 to tile 6) and change direction to down
    if[(t=4)&(d~0 1)&p[1]=N-1;
        /show "AB";
        :(@[TS 6;1;+;N-1+p 0];1 0)];
    
    / C->D 11 10 -> 7 1 (tile 5 to tile 2) and change direction to up
    if[(t=5)&(d~1 0)&p[0]=N-1;
        /show "CD";
        :(TS[2]+N-1+(0;p 1);-1 0)];

    if[(t=5)&(d~0 -1)&p[1]=0;
        /show "5l";
        :(TS[3]+N-1+(0;p 0);-1 0)];

    if[(t=3)&(d~1 0)&p[0]=N-1;
        show "3d";
        :(TS[5]+(N-1+p 1;0);0 1)];
    
    show "default";
    :@[pd;0;+;pd 1]mod(maxR;maxC); / inside 
 }

In [138]:
t3:test nextpd
all t3 .' (
    /(enlist (5 11;0 1);(8 14;1 0));
    /(enlist (4 11;0 1);(8 15;1 0));
    /(enlist (7 11;0 1);(8 12;1 0));
    /(enlist (11 10;1 0);(7 1;-1 0));
    /(enlist (11 8;1 0);(7 3;-1 0));
    /(enlist (11 11;1 0);(7 0;-1 0));
    enlist (enlist (7 5;1 0);(10 8;0 1))
 )

3 1
"3d"


1b


In [125]:
TS 3

4 4
